
# Feed Forward 
__1. All nodes are fully connected.__<br>
__2. Activation flows from input layer to output, without back loops.__ <Br>
__3. There is one layer between input.__<Br>
__4. [Towards Data Science Blog BY Andrew Tch for neural networks details ](https://towardsdatascience.com/the-mostly-complete-chart-of-neural-networks-explained-3fb6f2367464).__


### Titanic Dataset is used to implement Feed Forward
[Download Titanic Dataset](https://www.kaggle.com/c/titanic/data).


In [1]:
import numpy as np 
import pandas as pd


In [2]:
def reading_test_train_data():
    train = pd.read_csv('./datasets/train.csv')
    test = pd.read_csv('./datasets/test.csv')
    return train,test


def adding_title_using_name(dataset):
    """
    Fetching Mr,Miss,Mrs title from name 
    """ 
    dataset['Title'] = dataset['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    return dataset


def converting_title_to_int(title):
    """
    Assigning Number to each title
    """
    title= title.strip().lower()
    if title == 'mr':
        return 1
    elif title =='miss':
        return 2
    elif title =='mrs':
        return 3
    else:
        return 0
    
def age_into_grp(dataset):
    """
    converting age into age_group as bins
    """
    bins = [11,18,22,27,33,40,66, 120]
    labels = [str(i) for i in range(1,len(bins))]
    dataset['Age'] = pd.cut(dataset['Age'], bins, labels = labels,include_lowest = True)
    return dataset

In [3]:
train,test = reading_test_train_data()


In [4]:
# first 10 rows
train.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [5]:
# 891 rows 
# 12 columns /featurs including target
train.shape

(891, 12)

In [6]:
# only age count() is less then total df len means contains nan values
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


###  Manipulating --- AGE 

In [7]:
# filling age with age median
train['Age'] = train['Age'].fillna(train['Age'].median())

### ADDING NEW FEATURE 

In [8]:
# check who came with family or who came alone
train['with_family'] = train['SibSp'] + train['Parch']
train.loc[train['with_family'] > 0, 'not_alone'] = 0
train.loc[train['with_family'] == 0, 'not_alone'] = 1
train['not_alone'] = train['not_alone'].astype(int)
train.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,with_family,not_alone
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0,1


### Filling Embarked column

In [9]:
# checking Embarked data stats
train['Embarked'].describe()

count     889
unique      3
top         S
freq      644
Name: Embarked, dtype: object

In [10]:
train['Embarked'] = train['Embarked'].fillna('S')

In [11]:

train['Embarked'] = train['Embarked'].astype('category').cat.codes

### Manipulating FARE

In [12]:
train['Fare'].describe() 
train['Fare'] = train['Fare'].astype(int)

### Manipulating GENDER feature

In [16]:
train['Sex'].unique()

array(['male', 'female'], dtype=object)

In [13]:
train['Sex'] = train['Sex'].astype('category').cat.codes
train['Sex'].unique()

array([1, 0], dtype=int64)

### Manipulating Name and extracting Title from it
#### Then Converting them into category

In [14]:
train = adding_title_using_name(train)


In [15]:
train['Title'] = train['Title'].apply(converting_title_to_int)

In [16]:
train['Title'].unique()

array([1, 3, 2, 0], dtype=int64)

## converting age into bins - range

In [17]:
train = age_into_grp(train)

In [18]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,with_family,not_alone,Title
0,1,0,3,"Braund, Mr. Owen Harris",1,2,1,0,A/5 21171,7,NaN,2,1,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,5,1,0,PC 17599,71,C85,0,1,0,3
2,3,1,3,"Heikkinen, Miss. Laina",0,3,0,0,STON/O2. 3101282,7,NaN,2,0,1,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,5,1,0,113803,53,C123,2,1,0,3
4,5,0,3,"Allen, Mr. William Henry",1,5,0,0,373450,8,NaN,2,0,1,1


In [19]:
train.drop(['PassengerId','Name','Ticket','Cabin'],axis=1,inplace=True)

In [20]:
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,with_family,not_alone,Title
0,0,3,1,2,1,0,7,2,1,0,1
1,1,1,0,5,1,0,71,0,1,0,3
2,1,3,0,3,0,0,7,2,0,1,2
3,1,1,0,5,1,0,53,2,1,0,3
4,0,3,1,5,0,0,8,2,0,1,1


## HANDLING TEST DATA AND PREPARING IT


In [21]:
# viewing test dataset
test.head(10)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S
6,898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q
7,899,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,NaN,S
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,C
9,901,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,NaN,S


In [37]:
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


### Manipulting age for test dataset

In [22]:
# filling age with age median
test['Age'] = test['Age'].fillna(test['Age'].median())

### Creating With_familt and not_alone feature

In [23]:
# check who came with family or who came alone
test['with_family'] = test['SibSp'] + test['Parch']
test.loc[test['with_family'] > 0, 'not_alone'] = 0
test.loc[test['with_family'] == 0, 'not_alone'] = 1
test['not_alone'] = test['not_alone'].astype(int)
test.head(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,with_family,not_alone
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0,1
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,1,0
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0,1
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0,1
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,2,0


### Filling Embarked column in test dataset

In [25]:
# checking Embarked data stats
test['Embarked'].describe()

count     418
unique      3
top         S
freq      270
Name: Embarked, dtype: object

In [26]:
test['Embarked'] = test['Embarked'].fillna('S')
test['Embarked'] = test['Embarked'].astype('category').cat.codes

### Manipulating FARE  in test dataset

In [27]:
test['Fare'].describe() 
test['Fare'] = train['Fare'].astype(int)

### Manipulating gender feature in test dataset

In [29]:
test['Sex'] = test['Sex'].astype('category').cat.codes
test['Sex'].unique()

array([1, 0], dtype=int64)

### Manipulating Name and extracting Title from it in test dataset
#### Then Converting them into category

In [30]:
test = adding_title_using_name(test)
test['Title'] = test['Title'].apply(converting_title_to_int)
test['Title'].unique()

array([1, 3, 2, 0], dtype=int64)

### converting age into bins

In [31]:
test = age_into_grp(test)
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,with_family,not_alone,Title
0,892,3,"Kelly, Mr. James",1,5,0,0,330911,7,NaN,1,0,1,1
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",0,6,1,0,363272,71,NaN,2,1,0,3
2,894,2,"Myles, Mr. Thomas Francis",1,6,0,0,240276,7,NaN,1,0,1,1
3,895,3,"Wirz, Mr. Albert",1,3,0,0,315154,53,NaN,2,0,1,1
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,2,1,1,3101298,8,NaN,2,2,0,3


In [32]:
test.drop(['Name','Ticket','Cabin'],axis=1,inplace=True)

In [33]:
passengerId_test = test['PassengerId']

In [35]:
test.drop(['PassengerId'],axis=1,inplace=True)

###  Create feed forward neural network to train data